In [1]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
from sqlalchemy import create_engine

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 
#Original Sandbox Environment: 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'15',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '5867a73c-d596-4bbf-a45d-121167dff22c',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  #print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

#NOTE:
# I had to go in and put "jupyter notebook --NotebookApp.iopub_data_rate_limit=1e10"
# Into the Anaconda Prompt to change this to allow to pull data

# If that didn't work try using the local host URL as shown in the video

type(data)

dict

In [2]:
user = 'postgres'
password = '%40rydhamB383'
host = 'localhost'
port = '5432'
database = 'crypto_db'

# Create engine
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

In [3]:
import pandas as pd


#This allows you to see all the columns, not just like 15
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
#This normalizes the data and makes it all pretty in a dataframe

df = pd.json_normalize(data['data'])
df['timestamp'] = pd.to_datetime('now')
df

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,infinite_supply,platform,cmc_rank,self_reported_circulating_supply,self_reported_market_cap,tvl_ratio,last_updated,quote.USD.price,quote.USD.volume_24h,quote.USD.volume_change_24h,quote.USD.percent_change_1h,quote.USD.percent_change_24h,quote.USD.percent_change_7d,quote.USD.percent_change_30d,quote.USD.percent_change_60d,quote.USD.percent_change_90d,quote.USD.market_cap,quote.USD.market_cap_dominance,quote.USD.fully_diluted_market_cap,quote.USD.tvl,quote.USD.last_updated,platform.id,platform.name,platform.symbol,platform.slug,platform.token_address,timestamp
0,1,Bitcoin,BTC,bitcoin,12230,2010-07-13T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",2.100000e+07,1.988390e+07,1.988390e+07,False,NaN,1,NaN,NaN,None,2025-06-26T19:02:00.000Z,107462.327787,4.423686e+10,-13.6706,0.214149,-0.032732,2.876692,-2.141238,13.948781,28.315461,2.136771e+12,65.1655,2.256709e+12,None,2025-06-26T19:02:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-06-27 00:33:33.220159
1,1027,Ethereum,ETH,ethereum,10357,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,1.207183e+08,1.207183e+08,True,NaN,2,NaN,NaN,None,2025-06-26T19:02:00.000Z,2427.235285,1.811449e+10,7.0312,0.235727,0.263968,-3.176921,-9.288035,34.542634,29.295817,2.930116e+11,8.9360,2.930116e+11,None,2025-06-26T19:02:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-06-27 00:33:33.220159
2,825,Tether USDt,USDT,tether,134112,2015-02-25T00:00:00.000Z,"[stablecoin, asset-backed-stablecoin, usd-stab...",NaN,1.572172e+11,1.598262e+11,True,NaN,3,NaN,NaN,None,2025-06-26T19:01:00.000Z,1.000213,6.373690e+10,-8.1362,-0.004469,-0.014786,0.005217,-0.024048,-0.016784,0.096924,1.572507e+11,4.7970,1.598603e+11,None,2025-06-26T19:01:00.000Z,1027.0,Ethereum,ETH,ethereum,0xdac17f958d2ee523a2206206994597c13d831ec7,2025-06-27 00:33:33.220159
3,52,XRP,XRP,xrp,1632,2013-08-04T00:00:00.000Z,"[medium-of-exchange, enterprise-solutions, xrp...",1.000000e+11,5.900555e+10,9.998599e+10,False,NaN,4,NaN,NaN,None,2025-06-26T19:02:00.000Z,2.123363,2.897506e+09,3.0083,0.372456,-3.028933,-2.239717,-9.123896,-5.458017,-2.673930,1.252902e+11,3.8210,2.123363e+11,None,2025-06-26T19:02:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-06-27 00:33:33.220159
4,1839,BNB,BNB,bnb,2594,2017-07-25T00:00:00.000Z,"[marketplace, centralized-exchange, payments, ...",NaN,1.408857e+08,1.408857e+08,False,NaN,5,NaN,NaN,None,2025-06-26T19:02:00.000Z,645.668815,1.449573e+09,-11.2774,0.025172,-0.023151,0.497387,-6.318021,7.016532,3.868955,9.096547e+10,2.7742,9.096547e+10,None,2025-06-26T19:02:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-06-27 00:33:33.220159
5,5426,Solana,SOL,solana,940,2020-04-10T00:00:00.000Z,"[pos, platform, solana-ecosystem, cms-holdings...",NaN,5.343291e+08,6.038703e+08,True,NaN,6,5.252369e+08,7.435299e+10,None,2025-06-26T19:02:00.000Z,141.560866,3.159904e+09,2.0208,-0.167918,-1.265829,-2.673407,-20.712624,-5.541320,9.323308,7.564009e+10,2.3068,8.548440e+10,None,2025-06-26T19:02:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-06-27 00:33:33.220159
6,3408,USDC,USDC,usd-coin,28897,2018-10-08T00:00:00.000Z,"[medium-of-exchange, stablecoin, asset-backed-...",NaN,6.176305e+10,6.176305e+10,False,NaN,7,6.090122e+10,6.087562e+10,None,2025-06-26T19:02:00.000Z,0.999580,9.201797e+09,-10.2006,-0.023937,-0.012285,-0.026993,-0.022829,-0.039352,-0.047992,6.173708e+10,1.8828,6.173708e+10,None,2025-06-26T19:02:00.000Z,1027.0,Ethereum,ETH,ethereum,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,2025-06-27 00:33:33.220159
7,1958,TRON,TRX,tron,1190,2017-09-13T00:00:00.000Z,"[media, payments, tron-ecosystem, layer-1, dwf...",NaN,9.480593e+10,9.480597e+10,True,NaN,8,7.165966e+10,1.943027e+10,None,2025-06-26T19:02:00.000Z,0.271147,5.109894e+08,9.2426,0.180257,-0.391103,-1.221977,-1.451313,9.205525,16.162303,2.570630e+10,0.7840,2.570631e+10,None,2025-06-26T19:02:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-06-27 00:33:33.220159
8,74,Dogecoin,DOGE,dogecoin,1242,2013-12-15T00:00:00.000Z,"[mineable, pow, s

In [5]:
def api_runner():
    global df

    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 
    #Original Sandbox Environment: 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'15',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '5867a73c-d596-4bbf-a45d-121167dff22c',
    }
    
    session = Session()
    session.headers.update(headers)
    
    try:
      response = session.get(url, params=parameters)
      data = json.loads(response.text)
      #print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)


    df = pd.json_normalize(data['data'])
    df['timestamp'] = pd.to_datetime('now')
    

    if not os.path.isfile(r'D:\Crypto Website Automatic API pull\Cryptop_API.csv'):
        df.to_csv(r'D:\Crypto Website Automatic API pull\Cryptop_API.csv', header = 'column_names' ,index = False)
    else:
        df.to_csv(r'D:\Crypto Website Automatic API pull\Cryptop_API.csv', mode = 'a' , header = False ,index = False)

    df.to_sql( 'crypto_table' , engine , if_exists = 'append', index = False ) 
        

In [ ]:
import os 
from time import time
from time import sleep

for i in range(333):
    api_runner()
    print('API Runner completed')
    sleep(10800) #every 3 hour
exit()

API Runner completed


In [ ]:
df2 = pd.read_csv(r'D:\Crypto Website Automatic API pull\Cryptop_API.csv')
df2 = df2.drop(columns=['Unnamed: 0'])
df2